In [1]:
import pandas as pd
import numpy as np
from environment import Environment

In [2]:
from historical import HistoricalData
from pandas._libs.tslibs.timestamps import Timestamp, Timedelta

path = "data\\"
env_df = pd.read_csv(path + "Consolidated.csv", header = [0,1], index_col=[0])
targets_df = pd.read_csv(path + "Targets.csv", index_col=[0])
features_df = pd.read_csv(path + "Features.csv", header = [0,1], index_col=[0])


data = HistoricalData(env_df, features_df, targets_df)

In [3]:
import instrument
from historical import ShockMap, Distribution

date = Timestamp('2009-03-31')
#date = Timestamp('2019-03-31')
env_now = Environment(*data.get_env_args(date))

etfs = env_df[['EQ', 'FI', 'EM', 'RE']].columns.droplevel()
etf_dict = {etf: instrument.Equity(etf, 'USD') for etf in etfs}

sm = ShockMap(data, date)
etfs_cov = np.array(sm.targets_df[etfs].cov())

In [4]:
# Optimization
from scipy import optimize

N = len(etfs)
Q = etfs_cov
y0 = np.ones(N)/N

def f(y):
    return 0.5*np.dot(y, np.dot(Q, y)) - 0.1*sum(np.log(y))

def grad_f(y):
    return np.dot(Q, y) - 0.1/y

non_neg_constr = {'type':'ineq', 'fun':lambda x: np.dot(np.identity(N), x), 'jac':lambda x: np.identity(N)}

y = optimize.minimize(f, y0, jac=grad_f, constraints=non_neg_constr, tol=1e-7, options={'maxiter':500})
pos_dict = {}

In [5]:
# Optimization Result
y.nit, y.message

(115, 'Optimization terminated successfully.')

In [6]:
# Portfolio Weights
x = y.x/sum(y.x)
pd.Series(x, index=etfs)

SPY US Equity      0.010832
EFA US Equity      0.008230
XLF US Equity      0.007065
XLK US Equity      0.011272
XLV US Equity      0.013053
XLP US Equity      0.015555
XLE US Equity      0.012053
EWJ US Equity      0.010248
XLU US Equity      0.013977
XLI US Equity      0.008804
EZU US Equity      0.006712
XLB US Equity      0.008919
IXN US Equity      0.009669
IYZ US Equity      0.011860
IEV US Equity      0.007985
TIP US Equity      0.024901
AGG US Equity      0.042191
IEF US Equity      0.045785
TLT US Equity      0.021187
SHY US Equity      0.649911
LQD US Equity      0.020963
EWZ US Equity      0.007023
ISMUF US Equity    0.005458
ILF US Equity      0.007317
IYR US Equity      0.006621
RWR US Equity      0.006356
ICF US Equity      0.006053
dtype: float64

In [7]:
# Risk Contributions
np.multiply(x, np.dot(Q, x))/sum(np.multiply(x, np.dot(Q, x)).flatten())

array([0.03703172, 0.03703293, 0.03704184, 0.03704516, 0.03703188,
       0.03703881, 0.03703907, 0.03703489, 0.03703559, 0.03703963,
       0.03704092, 0.0370252 , 0.0370342 , 0.03702953, 0.03705069,
       0.03703899, 0.03702505, 0.03703866, 0.0370399 , 0.03705031,
       0.03703566, 0.03702898, 0.03703507, 0.0370303 , 0.03704228,
       0.03704405, 0.03703869])